In [1]:
import numpy as np
import re
import json
import pandas as pd

In [2]:
model = "text-davinci-001"

labels = pd.read_csv("results.txt", delimiter = ",").loc[:,["pairid", " varA","varB","dataset","groundtruth"]]
labels['pairid'] = labels['pairid'].str.extract('(\d{4})').astype(int) # convert pairid to int
labels["groundtruth"] = labels["groundtruth"].str.replace(r'\s+', '', regex=True) # remove white space in labels

weights = pd.read_csv("pairmeta.txt", delimiter = " ", header = None).iloc[:,[0,5]]
weights.rename(columns = {0:"pairid", 5:"weight"}, inplace=True)
weights['pairid'] = weights['pairid'].astype(int) # convert pairid to int

merged_df = labels.merge(weights, on='pairid')
merged_df["AtoB"] = ""
merged_df["BtoA"] = ""

with open(model + "_response.json", "r") as file:
    loaded_data = json.load(file)
    
merged_df.head()

,pairid,varA,varB,dataset,groundtruth,weight,AtoB,BtoA
0,1,Altitude,Temperature,DWD,->,0.166,,
1,2,Altitude,Precipitation,DWD,->,0.166,,
2,3,Longitude,Temperature,DWD,->,0.167,,
3,4,Altitude,Sunshine hours,DWD,->,0.166,,
4,5,Age,Length,Abalone,->,0.143,,


In [3]:
loaded_data[0]['pair0001,_res1']["choices"][0]["text"]

'\n\nYes'

In [4]:
def add_prediction(merged_df, loaded_data, model):
    merged_df = merged_df.set_index('pairid')
    for response in loaded_data:
        pairid, resid, answer = extract_answers(response)
        print(pairid,resid, answer)
#         if answer != 'A/B' and answer[0] != 'E' and (answer[0]=='A' or answer[0]=='B'):
#             answer = answer[0]
#         answer = '->' if answer == 'A' else '<-' if answer == 'B' else answer
        col = "AtoB" if resid == "1" else "BtoA"
        merged_df.loc[int(pairid),col] = answer
#         print("Converted:", answer, "\n")
    return merged_df


def extract_answers(s):
    # Compile regex patterns
    
    pairid_pattern = re.compile(r"\{'pair0(\d{3}),_res(\d)")
    
    # Search for pairid
    pairid_match = pairid_pattern.search(str(s))
    pairid = pairid_match.group(1) if pairid_match else None
    resid = pairid_match.group(2) if pairid_match else None
    
    # Search for answer
    ans = s["pair0"+ str(pairid) + ',_res' + str(resid)]["choices"][0]["text"]
    
    return pairid, resid, ans[2:]

In [5]:
results = add_prediction(merged_df, loaded_data, model)

001 1 Yes
001 2 Yes
002 1 Yes
002 2 Yes
003 1 Yes
003 2 Yes
004 1 Yes
004 2 Yes
005 1 Yes
005 2 Yes
006 1 Yes
006 2 Yes
007 1 Yes
007 2 Yes
008 1 Yes
008 2 Yes
009 1 Yes
009 2 Yes
010 1 Yes
010 2 Yes
011 1 Yes
011 2 Yes
012 1 Yes
012 2 Yes
013 1 Yes
013 2 Yes
014 1 Yes
014 2 Yes
015 1 Yes
015 2 Yes
016 1 Yes
016 2 Yes
017 1 Yes
017 2 Yes
018 1 Yes
018 2 Yes
019 1 Yes
019 2 Yes
020 1 Yes
020 2 Yes
021 1 Yes
021 2 Yes
022 1 Yes
022 2 Yes
023 1 Yes
023 2 Yes
024 1 Yes
024 2 Yes
025 1 Yes
025 2 Yes
026 1 Yes
026 2 Yes
027 1 Yes
027 2 Yes
028 1 Yes
028 2 Yes
029 1 Yes
029 2 Yes
030 1 Yes
030 2 Yes
031 1 Yes
031 2 Yes
032 1 Yes
032 2 Yes
033 1 Yes
033 2 Yes
034 1 Yes
034 2 Yes
035 1 Yes
035 2 Yes
036 1 Yes
036 2 Yes
037 1 Yes
037 2 Yes
038 1 Yes
038 2 Yes
039 1 Yes
039 2 Yes
040 1 Yes
040 2 Yes
041 1 Yes
041 2 Yes
042 1 Yes
042 2 Yes
043 1 Yes
043 2 Yes
044 1 Yes
044 2 Yes
045 1 Yes
045 2 Yes
046 1 Yes
046 2 Yes
047 1 Yes
047 2 Yes
048 1 Yes
048 2 Yes
049 1 Yes
049 2 Yes
050 1 Yes
050 2 Yes


In [6]:
results

,varA,varB,dataset,groundtruth,weight,AtoB,BtoA
pairid,,,,,,,
1,Altitude,Temperature,DWD,->,0.166,Yes,Yes
2,Altitude,Precipitation,DWD,->,0.166,Yes,Yes
3,Longitude,Temperature,DWD,->,0.167,Yes,Yes
4,Altitude,Sunshine hours,DWD,->,0.166,Yes,Yes
5,Age,Length,Abalone,->,0.143,Yes,Yes
...,...,...,...,...,...,...,...
104,time for passing 1. segment,time for passing 2. segment,D. Janzing,->,0.200,Yes,Yes
105,pixel vector of a patch,total brightness at the screen,D. Janzing,->,0.500,Yes,Yes
106,time required for one round,voltage,D. Janzing,<-,1.000,Yes,Yes


In [7]:
results["gt_AtoB"] = np.where(results["groundtruth"] == "->", "Yes", "No")
results["gt_BtoA"] = np.where(results["groundtruth"] == "->", "No", "Yes")
results["correct_AtoB"] = np.where(results["gt_AtoB"] == results["AtoB"], 1, 0)
results["correct_BtoA"] = np.where(results["gt_BtoA"] == results["BtoA"], 1, 0)
wtd_acc = (sum(results["correct_AtoB"]*results["weight"]) + sum(results["correct_BtoA"]*results["weight"])) / (results["weight"].sum()*2)
acc = (sum(results["correct_AtoB"]) + sum(results["correct_BtoA"])) / 108 / 2
print("acc:", acc, "weighted acc:", wtd_acc)

acc: 0.5 weighted acc: 0.5


In [8]:
results["correct"] = np.where((results["BtoA"] == results["gt_BtoA"]) & (results["gt_AtoB"] == results["AtoB"]), 1, 0)
wtd_acc = (sum(results["correct"]*results["weight"])) / (results["weight"].sum())
acc = (sum(results["correct"])) / 108
print("if we count both correct as correct: ")
print("acc:", acc, "weighted acc:", wtd_acc)

if we count both correct as correct: 
acc: 0.0 weighted acc: 0.0


In [9]:
results

,varA,varB,dataset,groundtruth,weight,AtoB,BtoA,gt_AtoB,gt_BtoA,correct_AtoB,correct_BtoA,correct
pairid,,,,,,,,,,,,
1,Altitude,Temperature,DWD,->,0.166,Yes,Yes,Yes,No,1,0,0
2,Altitude,Precipitation,DWD,->,0.166,Yes,Yes,Yes,No,1,0,0
3,Longitude,Temperature,DWD,->,0.167,Yes,Yes,Yes,No,1,0,0
4,Altitude,Sunshine hours,DWD,->,0.166,Yes,Yes,Yes,No,1,0,0
5,Age,Length,Abalone,->,0.143,Yes,Yes,Yes,No,1,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...
104,time for passing 1. segment,time for passing 2. segment,D. Janzing,->,0.200,Yes,Yes,Yes,No,1,0,0
105,pixel vector of a patch,total brightness at the screen,D. Janzing,->,0.500,Yes,Yes,Yes,No,1,0,0
106,time required for one round,voltage,D. Janzing,<-,1.000,Yes,Yes,No,Yes,0,1,0
